In [1]:
#import torch
#from torch import nn
#import numpy as np
#from torchsummary import summary
#import Lossv2

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#summary(Lossv2.Loss().to(device), (6, 768, 512))  ###  IDK why, but I have to switch the dims in AvgPool on Loss() to get this to work :/ But that breaks the stuff bellow

In [2]:
import shutil
import torch
import torch.nn as nn
import numpy as np
torch.manual_seed(100)
np.random.seed(100)
import Lossv2
import generateLossImages
import time

global counter 
counter = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def save_ckp(state, is_best, checkpoint_dir="./models/rest/", best_model_dir="./models/best/"):
    global counter 
    f_path = checkpoint_dir + str(counter) + '_checkpoint.pt'
    counter = counter + 1
    torch.save(state, f_path)
    if is_best:
        best_fpath = best_model_dir + 'best_model.pt'
        shutil.copyfile(f_path, best_fpath)

def load_ckp(model, optimizer, checkpoint_fpath="./models/best/best_model.pt"):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch'], checkpoint['index'], checkpoint['min_lr'], checkpoint['max_lr'], checkpoint['steps'], checkpoint['step_size'], checkpoint['falling'], checkpoint['threshold']

In [3]:
# in PyTorch 1.12 and later.
torch.backends.cuda.matmul.allow_tf32 = True

# The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
torch.backends.cudnn.allow_tf32 = True

loss_fn = nn.L1Loss()
min_lr = 0.0003
max_lr = 0.001
decay = 0.9
steps = 500
falling = True
start_epoch = 0
start_index = 0
momentum = 0.94
threshold = [0.16, 0.12, 0.09, 0.06, 0.04, 0.03, 0.02, 0.015, 0.01 -1]
step_size = (max_lr-min_lr)/steps
model = Lossv2.Loss().to(device).to(memory_format=torch.channels_last)
#x = torch.rand(22, 8, 512, 768).to(device).to(memory_format=torch.channels_last)
model = torch.jit.script(model)
optimizer = torch.optim.SGD(model.parameters(), lr=max_lr, momentum=momentum)
scaler = torch.cuda.amp.GradScaler(enabled=True)
#
#torch.autograd.set_detect_anomaly(False)


In [4]:
#loss_fn = nn.L1Loss()
#model = Lossv2.Loss().to(device)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.96)
#model, optimizer, start_epoch, start_index, min_lr, max_lr, steps, step_size, falling, threshold = load_ckp(model, optimizer)

In [5]:
#li = [31.007527351379395, 0.01501011848449707, 0.0010006427764892578, 28.98952603340149, 0.48702239990234375, 47.146605014801025, 12.32689380645752, 0.5665144920349121]
#su = sum(li)
#print([100*l/su for l in li])


In [6]:
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler

printing = True
running_loss = 0.
last_loss = 0.
lis = []
 # might have to delete when loading model
threshold_decay = 0.2
flags = [True for _ in threshold]
record = 1
epochs = 100
times = [0]*8
batch_size= 22

torch.backends.cudnn.benchmark = True
torch.autograd.detect_anomaly(check_nan=False)
torch.autograd.set_detect_anomaly(False, check_nan=False)
torch.autograd.profiler.profile(enabled=False)
torch.autograd.profiler.emit_nvtx(enabled=False)
#torch.autograd.gradgradcheck(check_undefined_grad= False)
#torch.autograd.gradcheck(check_undefined_grad= False)
#torch.no_grad()

scaler = GradScaler()
import warnings 
warnings.filterwarnings("ignore")

C:\Users\Rani\AppData\Local\Temp\ipykernel_19104\2895240763.py:17: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  torch.autograd.detect_anomaly(check_nan=False)


In [7]:
torch.set_grad_enabled(True)
for epoch in range(start_epoch, epochs):
    training = generateLossImages.MakeIter(start_index=start_index if epoch == start_epoch else 0, epoch=epoch) # start=2*71*50*4))#start_index if epoch == start_epoch else 0)) 
    #val = generateLossImages.get_image_pairs_transforms_with_loss("C:/Users/Rani/Desktop/ai_val/16")
    training_loader = torch.utils.data.DataLoader(training, batch_size=batch_size, num_workers=2)#), worker_init_fn=worker_init_fn) # num_workers
    end = 0
    before_start = 0
    in_upstart = True
    for index, data in enumerate(training_loader):#[90.06174230575562, 0.03202486038208008, 0.016014575958251953, 84.36967897415161, 1.3647308349609375, 137.4005196094513, 34.81299662590027, 1.9427943229675293, 0, 0]
        #print(len(inputs))
        before_start += 1
        #if before_start > 55*11/batch_size and in_upstart:
            #in_upstart = False
            #times = [0]*8
        #start = time.time()
        #if end != 0:
            #times[0] += (start-end)
        inputs, labels = data
        #labels = torch.unsqueeze(labels, dim=-1)
        inputs.to(memory_format=torch.channels_last)
        #load = time.time()
        #times[1] += load-start

        #lab = time.time()
        #times[2] += (lab-load)

        # Make predictions for this batch
        with autocast():
            outputs = model(inputs)

            #out = time.time()
            #times[3] += (out-lab)

            # Compute the loss and its gradients
            loss = loss_fn(outputs, labels)

            #los = time.time()
            #times[4] += (los-out)

        loss.backward()#scaler.scale(loss).backward()#loss.backward()
        
       # back = time.time()
        #times[5] += (back-los)
        # Gather data and report
        #running_loss += loss.item()
        
        #runn = time.time()
        #times[6] += (runn-back)
        optimizer.step()#scaler.step(optimizer)#optimizer.step()
        #scaler.update()
        optimizer.zero_grad(set_to_none=True)
        if falling:
            optimizer.param_groups[-1]['lr'] = optimizer.param_groups[-1]['lr'] - step_size
            if optimizer.param_groups[-1]['lr'] < min_lr:
                falling = False
                max_lr *= decay
                min_lr *= decay
                steps /= decay
                step_size = (max_lr-min_lr)/steps

                if printing:
                    print(min_lr)
                    print(max_lr)
                    print(steps)
                    print(step_size)
                checkpoint = {
                    'epoch': epoch,
                    'index': index,
                    'min_lr': min_lr,
                    'max_lr': max_lr,
                    'steps': steps,
                    'step_size': step_size,
                    'falling': falling,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'threshold': threshold
                }
                save_ckp(checkpoint, True)

        else: 
            optimizer.param_groups[-1]['lr'] += step_size
            if optimizer.param_groups[-1]['lr'] > max_lr:
                falling = True

        if printing and index % 300 == 9:
            print("loss")
            print(loss.item())
            print("lr")
            print(optimizer.param_groups[-1]['lr'])
            print("pred")
            print(labels)
            print(outputs)
        #        print('  batch {} loss: {}'.format(index + 1, last_loss))
        #        print(lis)
        #        lis = []
        #    running_loss = 0

        #    if last_loss < record:
        #        record = last_loss
        #        if last_loss < 0.08:
        #            checkpoint = {
        #            'epoch': epoch,
        #            'index': index,
        #            'min_lr': min_lr,
        #            'max_lr': max_lr,
        #            'steps': steps,
        #            'step_size': step_size,
        #            'falling': falling,
        #            'state_dict': model.state_dict(),
        #            'optimizer': optimizer.state_dict(),
        #            'threshold': threshold
        #        }
        #            save_ckp(checkpoint, True)
        #   print(times)
            
        #end = time.time()
        #times[7] += (end-runn)
        
                

loss
0.41122591495513916
lr
0.0009860000000000008
pred
tensor([0.0010, 0.9990, 0.2000, 0.6000, 0.4000, 0.8000, 0.1000, 0.9000, 0.7000,
        0.3000, 0.5000, 0.0010, 0.9990, 0.2000, 0.6000, 0.4000, 0.8000, 0.1000,
        0.9000, 0.7000, 0.3000, 0.5000], device='cuda:0')
tensor([[0.5806],
        [1.0000],
        [0.7974],
        [0.9849],
        [0.8696],
        [1.0000],
        [0.5796],
        [1.0000],
        [0.9985],
        [0.6792],
        [0.9331],
        [0.6143],
        [1.0000],
        [0.6299],
        [0.9883],
        [0.8896],
        [1.0000],
        [0.5796],
        [1.0000],
        [0.9995],
        [0.6240],
        [0.9473]], device='cuda:0', dtype=torch.float16,
       grad_fn=<DifferentiableGraphBackward>)
loss
0.2733413875102997
lr
0.0005659999999999912
pred
tensor([0.0010, 0.9990, 0.2000, 0.6000, 0.4000, 0.8000, 0.1000, 0.9000, 0.7000,
        0.3000, 0.5000, 0.0010, 0.9990, 0.2000, 0.6000, 0.4000, 0.8000, 0.1000,
        0.9000, 0.7000, 0.3000, 

OSError: [Errno 22] Invalid argument: './models/best/best_model.pt'

In [ ]:
print(times)
su = sum(times)
print([100*l/su for l in times])


[0.004003763198852539, 0.04904508590698242, 0.0, 0.16615056991577148, 0.001001119613647461, 1.1347110271453857, 0.0, 0.027023792266845703]
[0.2897214529997548, 3.5490144760661924, 0.0, 12.02303486543126, 0.07244330263478656, 82.11028254958329, 0.0, 1.9555033532847146]


In [ ]:
print(su/15*6645*50*4/86400)

1.4171235268866573
